# """
    THIS SCRIPT PERFORM PHOSCPREDICTION FROM IMAGE CROPS
    
    1. THE IMAGE CROPS CAN BE FROM A YOLOV3 PREDICTION SCRIPT 
    2. THE IMAGE CROPS CAN BE FROM A ORIGINAL BB FROM LOCALIZATION DATA
    3. THIS SCRIPT PROJECT BACK THE RESULT ON ORIGINAL IMAGE WHERE WE
    CAN SEE CORRECT AND WRONG PREDICTION.
    4. IMAGE CROPS HAS SPECIFIC NAME FORMAT
    
"""

In [ ]:
#! conda info
#import tensorflow as tf

! nvidia-smi

"""
This is PHOSCNET model prediction script
"""

In [1]:
from phoscPipeline import * # contains phosc model 
from phos_label_generator import gen_label
from phoc_label_generator import gen_phoc_label
import os

2022-05-04 09:58:55.970359: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-04 09:58:55.970378: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#! conda install numpy -y
#!conda install -c anaconda numpy -y
#! conda update -n base -c defaults conda -y
#! conda install -c anaconda jupyter_client -y
#! conda install -c anaconda jupyter_core -y


In [2]:
model=build_model()
MODEL="./new_1_256_"
model.load_weights(MODEL+".h5")
#model=tf.keras.models.load_model(MODEL+".h5")
#print(MODEL)

2022-05-04 09:59:00.134868: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-04 09:59:00.135231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-04 09:59:00.135296: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-04 09:59:00.135337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-04 09:59:00.147950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [4]:
#print(model.summary())

In [3]:
"""
    returns a dictionary having word as a key and PHOSC representation as a value 
"""

def get_comb_label(x):
    phos_labels=gen_label(x)
    phoc_labels=gen_phoc_label(x)
    test_labels=dict()
    for x in phos_labels:
        test_labels[x]=np.concatenate((phos_labels[x],phoc_labels[x]),axis=0)
    return test_labels

#train_word_label=get_comb_label(list(set(df_lex['Word'])))

def similarity(x,y):
    return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


"""
    below predict word from  image crop  image  using PHOSCNET 
"""

In [4]:
def predict(path2Image,test_word_label):
    
    x=img_to_array(load_img(path2Image,target_size=(50,250)))

    x = np.expand_dims(x, axis=0)
    y_pred=model.predict(x)
    y_pred=np.squeeze(np.concatenate((y_pred[0],y_pred[1]),axis=1))
    #print(y_pred)
    mx=0
    for k in test_word_label:
        temp=similarity(y_pred,test_word_label[k])
        if temp>mx:
            mx=temp
            op=k
            
    return op,mx


In [5]:
testImagesList=os.listdir("./crops/")
len(testImagesList)

10

In [6]:
"""
    gather all words corresponding to specific single image

"""

import re,os

#images=['c04-139.png', 'g06-018r.png', 'a01-072x.png', 'h01-010.png', 'c04-050.png', 'm01-032.png', 'g06-042r.png', 'g06-026i.png', 'h07-080.png', 'e04-022.png','g06-047h.png']

images=["m01-084.png",'m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

orgImage=images[0]

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/"

#testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/"


def getPhoscFromText(testImgPath,orgImage):
    
    """
        input:image directory and image name containing image crops 
        output: dictionary having word as key and PHOSC representation as value
        
        working: input contains paths to dictionary containing all crops of words present in image
        from labels of image in folder, original word labels are extracted and from it PHOSCNET representation 
        is derived
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,imgName in enumerate(os.listdir(testImgPath+orgImage)):

        word=imgName.split("]_")[1]
        word=word.split("_.png")[0]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    #print(len(allWords))
    #allWords
    return test_word_label
    
    
#test_word_label=getPhoscFromText(testImgPath,orgImage)

In [ ]:
#"images=os.listdir("/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//")

In [ ]:
"""
    below part takes input single crop folder name and image name as a input
    and perform PHOSC prediction on it, It also plots the prediction on
    original image
"""

import cv2
import os

testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"+orgImage+"//"
savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3//cropPHOSCVisual//"
image=cv2.imread(testImgPath+orgImage)
print(" original image name:",orgImage," original image shape:",image.shape)

matchCount=0
missMatch=0

for indx,cropName in enumerate(os.listdir(testCropPath)):
    
    print("")
    print("cropName:",cropName)
    word=cropName.split("]_")[1]
    word=word.split("_.png")[0]
    word= regex.sub('', word)
    
    if len(word)<2: # skip 
        continue
    
    cor=cropName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    #imgName=imgName.split("_")[0]
    
    predWord,score=predict(testCropPath+cropName)
    
    print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2)
    
    if word==predWord:
        #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
        #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        matchCount+=1
    else:
        
        cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
        cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        missMatch+=1
    

cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
cv2.imwrite(savePath+orgImage,image)

print("match",matchCount)
print("miss match:",missMatch)
print(" Accuracy:",matchCount/(matchCount+missMatch))


In [ ]:
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"
os.listdir(testCropPath)


In [16]:
import cv2
import os

import re
regex = re.compile('[^a-zA-Z]')

#images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']
#images=['m01-084.png']

#orgImage=images[0]

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/dataset/forms//" #"/home/aniketag/Documents/phd/yolov5/data/datasets/forms/m01-084.png"
#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 

testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"

images=os.listdir(testCropPath)

savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3//cropPHOSCVisual1//"

#print(" images:",images[0] in os.listdir(testImgPath))




e04-095.png  is dir: True
46
c06-095.png  is dir: True
83
g06-045d.png  is dir: True
121
f07-013.png  is dir: True
158
g06-011g.png  is dir: True
208
k04-142.png  is dir: True
237
n04-213.png  is dir: True
276
a05-039.png  is dir: True
315
a02-000.png  is dir: True
339
a02-072.png  is dir: True
380
a01-117.png  is dir: True
411
e04-052.png  is dir: True
443
l01-082.png  is dir: True
469
d01-019.png  is dir: True
500
g07-042.png  is dir: True
528
c06-011.png  is dir: True
555
r06-106.png  is dir: True
581
a02-111.png  is dir: True
617
f07-076a.png  is dir: True
650
n02-049.png  is dir: True
670
b01-057.png  is dir: True
709
m04-113.png  is dir: True
738
c04-017.png  is dir: True
765
r06-057.png  is dir: True
794
p02-022.png  is dir: True
816
a02-020.png  is dir: True
839
k03-117.png  is dir: True
860
c03-007a.png  is dir: True
886
d04-028.png  is dir: True
908
d05-030.png  is dir: True
936
d06-025.png  is dir: True
952
d04-101.png  is dir: True
969
c06-138.png  is dir: True
978
c06-005.

KeyboardInterrupt: 

In [23]:
testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/dataset/forms//" #"/home/aniketag/Documents/phd/yolov5/data/datasets/forms/m01-084.png"
#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 

testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"

images=os.listdir(testCropPath)

savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3//cropPHOSCVisual1//"

print(" images:",images[0] in os.listdir(testImgPath))

allWords=[]

for imgNo,orgImage in enumerate(images):
    
    try:
        if imgNo%100==0:
            print(orgImage," is dir:",os.path.isdir(testCropPath+orgImage)," UNIQUE WORDS:",len(allWords)," tot:",len(images))

        #testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/"+orgImage
        #testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"#+orgImage+"//"


        if not os.path.isdir(testCropPath+orgImage):
            continue
        #print("&&")

        for indx1,imgName in enumerate(os.listdir(testCropPath+orgImage)):

            word=imgName.split("]_")[1]
            word=word.split("_.png")[0]
            word= regex.sub('', word)
            allWords.append(word)

        allWords=list(set(allWords))
        test_word_label=get_comb_label(allWords)
        #print(len(allWords))
    except Exception as e:
        pass


 images: True
e04-095.png  is dir: True  UNIQUE WORDS: 0  tot: 1306
g06-018h.png  is dir: True  UNIQUE WORDS: 2263  tot: 1306
f01-081.png  is dir: True  UNIQUE WORDS: 3678  tot: 1306
n06-140.png  is dir: True  UNIQUE WORDS: 4894  tot: 1306
f04-074.png  is dir: True  UNIQUE WORDS: 5752  tot: 1306
g01-043.png  is dir: True  UNIQUE WORDS: 6553  tot: 1306
g06-018k.png  is dir: True  UNIQUE WORDS: 7338  tot: 1306
n02-037.png  is dir: True  UNIQUE WORDS: 8047  tot: 1306
g07-022a.png  is dir: True  UNIQUE WORDS: 8788  tot: 1306
c03-096f.png  is dir: True  UNIQUE WORDS: 9451  tot: 1306
m03-110.png  is dir: True  UNIQUE WORDS: 9979  tot: 1306
b03-092.png  is dir: True  UNIQUE WORDS: 10584  tot: 1306
g01-016.png  is dir: True  UNIQUE WORDS: 11045  tot: 1306
c03-000a.png  is dir: True  UNIQUE WORDS: 11491  tot: 1306


"""

    below code predicts PHOSC and visualizes all the image present in testCrop folder 
    to add more images into it changes are required in file 
    mnist_testYolo.txt
    
"""

In [27]:
testCropPath="/home/aniketag/Documents/phd/PHOSC-Zero-Shot-Word-Recognition-main/data/newFilterData//cropsYoloV3Test"
testPhoscCrop=os.listdir(testCropPath)
len(testPhoscCrop)
testPhoscCrop[0]

'f07-039a.png_47_[1814, 1229, 1968, 1272]_new_.png'

In [32]:
"""
    gather all words
    
"""
import cv2
import os

import re
regex = re.compile('[^a-zA-Z]')

#images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']
#images=['m01-084.png']

#orgImage=images[0]

    
for imgNo,orgImage in enumerate(images):
    
    if imgNo%100==0:
        print("imgNo:",imgNo," total:",len(images))
    
    matchCount=0
    missMatch=0
    testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"
    
    
    if not os.path.isdir(testCropPath+orgImage):
        continue
    
    #test_word_label=getPhoscFromText(testCropPath,orgImage)

    testCropPath=testCropPath+orgImage+"//"

    if os.path.isfile(savePath+orgImage+"_singleDict_.png"):
        continue
    
    #print(" imageName:",orgImage," is file:",os.path.isfile(testImgPath+orgImage)," No of unique words:",len(allWords))

    image=cv2.imread(testImgPath+orgImage)
    #print(" original image shape:",image.shape)

    df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   

    
    for indx,cropImgName in enumerate(os.listdir(testCropPath)):

        #print(" cropImgName:",cropImgName)
        #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]

        word=cropImgName.split("]_")[1]
        word=word.split("_.png")[0]

        cor=cropImgName.split("[")[1]
        cor=cor.split("]")[0]
        cor="["+cor+"]"
        cor=eval(cor)

        x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])

        #imgName=imgName.split("_")[0]

        predWord,score=predict(testCropPath+cropImgName,test_word_label)

        #print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
        #print("")
                
        df.loc[indx,"image_name"]=orgImage
        df.loc[indx,"crop_name"]=cropImgName
        df.loc[indx,"original_word"]=word
        df.loc[indx,"predicted_word"]=predWord       
        
            

        if word.lower()==predWord.lower():
            #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            
            image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
            #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            matchCount+=1
        else:

            cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
            cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            missMatch+=1

            
        if cropImgName in testPhoscCrop:
            image = cv2.line(image, (x1, y1), (x2, y2),(255,0,0), 2)

            

    print("match",matchCount)
    print("miss match:",missMatch)
    print(" Accuracy:",matchCount/(matchCount+missMatch))

    cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
    cv2.imwrite(savePath+orgImage+"_singleDict_.png",image)
    df.to_csv(savePath+orgImage+"_singleDict_.csv")
    
    #input("check!!!")
    

imgNo: 0  total: 1306


/tmp/ipykernel_3610/1231027792.py:16: RuntimeWarning: invalid value encountered in double_scalars
  return 1000*np.dot(x,y)/(LA.norm(x)*LA.norm(y))


match 61
miss match: 6
 Accuracy: 0.9104477611940298
match 86
miss match: 10
 Accuracy: 0.8958333333333334
match 59
miss match: 11
 Accuracy: 0.8428571428571429
match 66
miss match: 4
 Accuracy: 0.9428571428571428
match 60
miss match: 18
 Accuracy: 0.7692307692307693
match 43
miss match: 10
 Accuracy: 0.8113207547169812
match 71
miss match: 10
 Accuracy: 0.8765432098765432
match 67
miss match: 19
 Accuracy: 0.7790697674418605
match 57
miss match: 13
 Accuracy: 0.8142857142857143
match 48
miss match: 22
 Accuracy: 0.6857142857142857
match 59
miss match: 10
 Accuracy: 0.855072463768116
match 69
miss match: 11
 Accuracy: 0.8625
match 63
miss match: 5
 Accuracy: 0.9264705882352942
match 63
miss match: 6
 Accuracy: 0.9130434782608695
match 64
miss match: 15
 Accuracy: 0.810126582278481
match 60
miss match: 8
 Accuracy: 0.8823529411764706
match 48
miss match: 5
 Accuracy: 0.9056603773584906
match 80
miss match: 12
 Accuracy: 0.8695652173913043
match 71
miss match: 21
 Accuracy: 0.77173913043

match 57
miss match: 9
 Accuracy: 0.8636363636363636
match 69
miss match: 7
 Accuracy: 0.9078947368421053
match 63
miss match: 8
 Accuracy: 0.8873239436619719
match 53
miss match: 5
 Accuracy: 0.9137931034482759
match 55
miss match: 23
 Accuracy: 0.7051282051282052
match 74
miss match: 6
 Accuracy: 0.925
match 57
miss match: 7
 Accuracy: 0.890625
match 70
miss match: 3
 Accuracy: 0.958904109589041
match 52
miss match: 4
 Accuracy: 0.9285714285714286
match 62
miss match: 6
 Accuracy: 0.9117647058823529
match 66
miss match: 7
 Accuracy: 0.9041095890410958
match 57
miss match: 7
 Accuracy: 0.890625
match 47
miss match: 10
 Accuracy: 0.8245614035087719
match 62
miss match: 15
 Accuracy: 0.8051948051948052
match 72
miss match: 8
 Accuracy: 0.9
match 36
miss match: 22
 Accuracy: 0.6206896551724138
match 73
miss match: 11
 Accuracy: 0.8690476190476191
match 51
miss match: 21
 Accuracy: 0.7083333333333334
match 56
miss match: 9
 Accuracy: 0.8615384615384616
match 70
miss match: 9
 Accuracy: 0.

match 47
miss match: 8
 Accuracy: 0.8545454545454545
match 72
miss match: 14
 Accuracy: 0.8372093023255814
match 65
miss match: 11
 Accuracy: 0.8552631578947368
match 52
miss match: 14
 Accuracy: 0.7878787878787878
match 68
miss match: 18
 Accuracy: 0.7906976744186046
match 52
miss match: 8
 Accuracy: 0.8666666666666667
match 78
miss match: 1
 Accuracy: 0.9873417721518988
match 56
miss match: 18
 Accuracy: 0.7567567567567568
match 77
miss match: 10
 Accuracy: 0.8850574712643678
match 74
miss match: 16
 Accuracy: 0.8222222222222222
match 46
miss match: 15
 Accuracy: 0.7540983606557377
match 50
miss match: 13
 Accuracy: 0.7936507936507936
match 83
miss match: 8
 Accuracy: 0.9120879120879121
match 70
miss match: 10
 Accuracy: 0.875
match 59
miss match: 8
 Accuracy: 0.8805970149253731
match 73
miss match: 11
 Accuracy: 0.8690476190476191
match 78
miss match: 8
 Accuracy: 0.9069767441860465
match 61
miss match: 8
 Accuracy: 0.8840579710144928
match 83
miss match: 8
 Accuracy: 0.912087912087

KeyboardInterrupt: 

In [31]:
#print(allWords)
#len(allWords)
#images

s="Aniket"
s.lower()

'aniket'

In [11]:
len(images),len(allWords)
testCropPath,os.path.isdir(testCropPath+orgImage),testCropPath+orgImage

('/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//',
 True,
 '/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//a02-067.png')

"""

    BELOW PART CAN BE USED TO VISUALIZE THE PERFORMANCE OF ORIGINAL CROPS,
    REQUIRED CROPS ARE TAKEN FROM PHOSCNET TRAINING DATA.
    THE CROP NAMES DOES NOT INCLUDE WORDS PRESENT INSIDE IT OR COORDINATE TO HANDLE IT
    CSV data_444_april.csv IS USED WHICH HAS REQUIRED INFORMATION

"""

"""

    1. Select test image_name from data_444*.csv and fetch all records related to that.
    
    2. From fetched records get BB and text of location IE create a new dataframe which has BB and corresponding text
    * there is difference between image name and crop name
    
    3. From PHOSCNET test data separate all crops corresponding to above recods and create a separate folder for it,
    here we have only Image name from 1,2 but not corresponding crop name so we need to derive
    it with string matching.
    
    Get the PHOSC vector space for all word labels present in speecific image
    
    4. Pass those crop to PHOSCNET get prediction and visualize it..
    
"""

In [ ]:
import pandas as pd
import os
import cv2
import re

"""
    allPaths
"""

csvFile="/home/aniketag/Documents/phd/yolov5/data/datasets/data_444_april.csv"
phoscTestImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/phoscTest//"
allPhoscTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/IAM_test/"
PhoscOrgTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"

In [ ]:

df1=pd.read_csv(csvFile)
print(df1.shape)

phoscTestImg=os.listdir(phoscTestImgPath)
print(" total test images:",len(phoscTestImg))


In [ ]:
"""
1,2.
    input: image name for which BB,Text records are needed
    output: dataframe with relevant information
    
"""

def fetchRecord(df,imageName):
    
    temp=df[df["image_name"]==imageName]
    print(" No of records:",temp.shape)
    return temp


"""
3.
    crops corresponding to specific image file
    a. get list of all crops used for PHOSCNET testing
    b. filter crops necessary for specific image file
    
"""
from shutil import copy2

def filterCrop(allCropList,imageName):
    
    imgCropList=[]
    
    imageName=imageName.split(".png")[0]
    
    cropPhoscTestData="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/cropPhoscTestData//"
    basePhoscData="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/IAM_test//"
    
    for cropName in allCropList:
        
        if imageName in cropName:
            
            imgCropList.append(cropName)

    print(" TOTAL CROPS CORRESPONDING TO IMAGE "+imageName+" ARE:",len(imgCropList))
    
    
    try:
        os.mkdir(cropPhoscTestData+imageName)
    except Exception as e:
        pass
    
    for filterCropName in imgCropList:
        copy2(basePhoscData+filterCropName,cropPhoscTestData+imageName)
        
        
    print(" transferred crop:",len(cropPhoscTestData+imageName)) 
        
    return imgCropList

#imageName="m01-125.png"
#allCropList=os.listdir(allPhoscTestCrops)
#imgCropList=filterCrop(allCropList,imageName)
#imgCropList

def getPhoscFromDF(df):
    
    """
        input: Dataframe containing all image information like text,coordinate etc.
        output: dictionary having word as key and PHOSC representation as value
        
        working: similar to getPhoscFromText()
        
    """
    
    regex = re.compile('[^a-zA-Z]')
    allWords=[]


    for indx,row in df.iterrows():

        word=row["text"]
        word= regex.sub('', word)
        allWords.append(word)

    allWords=list(set(allWords))
    test_word_label=get_comb_label(allWords)
    print(len(allWords))
    #allWords
    return test_word_label

imageName="m01-084.png"
temp=fetchRecord(df1,imageName)
#temp.columns
#test_word_label=getPhoscFromDF(temp)
#temp.head(5)

"""
    save crops from original coordinates
    As there is no way to map PHOSCNET test data crops to original image 
    
    below function extracts crops from original image, 
    The crops are saved in following format ImgName.png_IndexNumber_[x1,y1, x2, y2]_textString_.png
    
    The crop coordinate are from original dataset It is not predicted by YOLOV3
    
"""

def PhoscOrgCrops(df,imgName):
    
    image=cv2.imread(phoscTestImgPath+imgName)
    
    print(image.shape)
    
    regex = re.compile('[^a-zA-Z]')
    
    
    try:
        os.mkdir(PhoscOrgTestCrops+imgName)
    
    except Exception as e:
        pass
    
    for indx,row in df.iterrows():
        #print(row)
        word=row["text"]
        word= regex.sub('', word)
        
        if len(word)<2:
            continue

        x1,y1,x2,y2=row["org_x1"],row["org_y1"],row["org_x2"],row["org_y2"]
        
        tempName=imgName+"_"+str(indx)+"_"+str([x1,y1,x2,y2])+"_"+word+".png"
        
        cropImage=image[y1:y2,x1:x2]
        #print(cropImage.shape)

        cv2.imwrite(PhoscOrgTestCrops+imgName+"//"+tempName,cropImage)
        


#PhoscOrgCrops(temp,imageName)



In [ ]:
imageName

In [ ]:
"""
    gather all words
"""
import cv2
import os

import re
regex = re.compile('[^a-zA-Z]')

images=['m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']
#images=['m01-084.png']

#orgImage=images[0]

#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" #"/home/aniketag/Documents/phd/yolov5/data/datasets/forms/m01-084.png"
#testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 

#testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"
PhoscOrgTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"

images=os.listdir(phoscTestImgPath)

savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3//cropPHOSCVisual//"

print(" images:",images[0] in os.listdir(testImgPath))


for imgNumber,orgImage in enumerate(images):
    
    try:
        print("")
        
        if os.path.isfile(savePath+orgImage+"_originalCrops_.png"):
            continue

        matchCount=0
        missMatch=0
        PhoscOrgTestCrops="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crop2//"

        temp=fetchRecord(df1,orgImage) # get relevant records

        PhoscOrgCrops(temp,orgImage) # create crops

        test_word_label=getPhoscFromText(PhoscOrgTestCrops,orgImage) # get attribute space

        PhoscOrgTestCrops=PhoscOrgTestCrops+orgImage+"//"


        #print(" imageName:",orgImage," No of unique words:",len(allWords)," is file:",os.path.isfile(testImgPath+orgImage))

        print(" imageName:",orgImage," No of unique words:",len(test_word_label.keys())," is file:",os.path.isfile(phoscTestImgPath+orgImage))


        image=cv2.imread(phoscTestImgPath+orgImage)
        #print(" original image shape:",image.shape)

        df=pd.DataFrame(index=range(200),columns=["image_name","crop_name","original_word","predicted_word"])   

        for indx,cropImgName in enumerate(os.listdir(PhoscOrgTestCrops)):

            #print(" cropImgName:",cropImgName)

            #temp=pd.DataFrame(columns=["image_name","crop_name","original_word","predicted_word"]

            word=cropImgName.split("]_")[1]
            word=word.split(".png")[0]

            cor=cropImgName.split("[")[1]
            cor=cor.split("]")[0]
            cor="["+cor+"]"
            cor=eval(cor)

            x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])

            #imgName=imgName.split("_")[0]

            predWord,score=predict(PhoscOrgTestCrops+cropImgName,test_word_label)
            predWord=predWord[:-3]
            #print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
            #print("")

            df.loc[indx,"image_name"]=orgImage
            df.loc[indx,"crop_name"]=cropImgName
            df.loc[indx,"original_word"]=word
            df.loc[indx,"predicted_word"]=predWord        

            if word==predWord:
                #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
                #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                matchCount+=1
            else:

                cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
                cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
                missMatch+=1


        print("match",matchCount)
        print("miss match:",missMatch)
        print(" Accuracy:",matchCount/(matchCount+missMatch))

        cv2.putText(image,str([matchCount,missMatch,matchCount/(matchCount+missMatch)]), (15,100), cv2.FONT_HERSHEY_SIMPLEX, 3, 255)
        cv2.imwrite(savePath+orgImage+"_originalCrops_.png",image)
        df.to_csv(savePath+orgImage+"_originalCrops_.csv")
    
    except Exception as e:
        pass
    
    #input("check!!!")
    

In [ ]:
len(os.listdir(phoscTestImgPath))

In [ ]:
df1=pd.read_csv("/home/aniketag/Documents/phd/yolov5/data/datasets/data_444_april.csv")
df1.shape

temp=df1[df1["image_name"]=="m01-084.png"]
temp.shape
temp.head(10)

In [ ]:
"""
    gather all words
"""

import re,os
regex = re.compile('[^a-zA-Z]')
allWords=[]

#images=['c04-139.png', 'g06-018r.png', 'a01-072x.png', 'h01-010.png', 'c04-050.png', 'm01-032.png', 'g06-042r.png', 'g06-026i.png', 'h07-080.png', 'e04-022.png','g06-047h.png']

images=["m01-084.png"]#,"m01-084.png",'m01-090.png', 'm01-110.png', 'm01-049.png', 'm01-060.png', 'm01-104.png', 'm01-095.png', 'm01-121.png', 'm01-115.png', 'm01-084.png', 'm01-079.png']

orgImage=images[0]

#testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops/"+orgImage

testImgPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/data/IAM_Data/"+orgImage

for indx,rows in temp.iterrows():
    
    word=rows["text"]
    word= regex.sub('', word)

    if len(word)>1:
        allWords.append(word)
    
allWords=list(set(allWords))
test_word_label=get_comb_label(allWords)
len(allWords)
#allWords

In [ ]:
import cv2
import os


testImgPath="/home/aniketag/Documents/phd/yolov5/data/datasets/forms//" 
testCropPath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/crops//"+orgImage+"//"
savePath="/home/aniketag/Documents/phd/TensorFlow-2.x-YOLOv3/final//"
image=cv2.imread(testImgPath+orgImage)
print(" original image shape:",image.shape)

matchCount=0
missMatch=0

for indx,imgName in enumerate(os.listdir(testCropPath)):
    
    print(" imgName:",imgName)
    word=imgName.split("]_")[1]
    word=word.split("_.png")[0]
    
    cor=imgName.split("[")[1]
    cor=cor.split("]")[0]
    cor="["+cor+"]"
    cor=eval(cor)
    
    x1,y1,x2,y2=int(cor[0]),int(cor[1]),int(cor[2]),int(cor[3])
        
    #imgName=imgName.split("_")[0]
    
    
    predWord,score=predict(testCropPath+imgName)
    
    print(" word:",word," ",predWord," score:",score," coordinate:",x1,y1,x2,y2,)
    print("")
    
    if word==predWord:
        #cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,255,0), 3)
        #cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        matchCount+=1
    else:
        
        cv2.putText(image,word, (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        image = cv2.rectangle(image, (x1, y1), (x2, y2),(0,0,255), 3)
        cv2.putText(image,predWord, (x1,y2+50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
        missMatch+=1
        
cv2.imwrite(savePath+orgImage,image)

print("match",matchCount)
print("miss match:",missMatch)
print(" Accuracy:",matchCount/(matchCount+missMatch))


In [ ]:
import os
os.getcwd()